# Tectonic Model: saving, reloading, checkpointing

Basic checkpointing is acheived by converting TectonicModel objects to dictionary, then pickling. 

UWSubduction includes a small class called checkpoint that helps organise the checkponting. Currently, it has teh capacity to handle checkpinting of:

* underworld2 mesh and swarm objects objects (`cp.addObject`)
* Python dictionaries (`cp.addDict`)

Where `cp` is a checkpoint object

When we save a checkpoint a TectonicModel, the dictionary only stores the underlying Networkx graph, containing information about plates locatipions (nodes), boundaries (egdes) and velocities.

This means we always need to rebuild the TectonicModel object every time the Uw model is run. The saved data gets loaded by using the `pop_from_dict_of_lists` method.

In [1]:
import numpy as np
import os
import pickle
import underworld as uw
from underworld import function as fn
import glucifer
from easydict import EasyDict as edict
import networkx as nx
import operator
import UWsubduction as usub

In [2]:
from UWsubduction.utils import checkpoint

#create a checkpoint object
cp = checkpoint(savepath = './checkpoint')

startFresh = False #if this false we'll load the TectonicModel from a checkpoint (a pickled dictionary)

/opt/UWsubduction/UWsubduction/utils/__init__.py:9: UserWarning: 

The easymodels module is not supported.
Questions should be addressed to sandd@student.unimelb.edu.au 
 
  Questions should be addressed to sandd@student.unimelb.edu.au \n """


could no load saved state info.


In [4]:
#if restart, attempt to read in saved dicts. 
with open(os.path.join('./checkpoint/0/', 'tmDict.pkl'), 'rb') as fp:
        tmDict = pickle.load(fp)
            
tmDict.keys() 

dict_keys([1, 2, 3])

## Setup a toy model

In [5]:


mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                 elementRes  = (96, 48), 
                                 minCoord    = (-1.0, 0.0), 
                                 maxCoord    = (1.0, 1.0) )


temperatureField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )
velocityField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=2 )
coordinate = fn.input()


In [6]:
cp.restart

True

In [7]:
#tectonic model, 3 plates


tm = usub.TectonicModel(mesh, 0, 20, 1)

if startFresh is True:
    print('starting fresh')
    
    tm.add_plate(1, velocities = 1)
    tm.add_plate(2,velocities = -1)
    tm.add_plate(3, velocities = 0)

    tm.add_left_boundary(1)
    tm.add_subzone(1, 2, 0.25, subInitAge=50.0, upperInitAge=10.0)
    tm.add_ridge(2, 3, 0.5)
    tm.add_right_boundary(3, 30.)
    
else:
    print('loading from checkpoint / dict')
    tm.pop_from_dict_of_lists(tmDict)

loading from checkpoint / dict


## Checkpointing

The TectonicModel object (tm) is a dictionary-like structure. Here we create a conversion to a dictionary-of-dictionaries and add this to the checkpoint object this provides an easy way to save/load the TectonicModel object n the case of restarts. 

* The object `tmDict` will remain updated as we update the TectonicModel (tm)

In [8]:
tmDict = nx.to_dict_of_dicts(tm)
cp.addDict(tmDict, 'tmDict')

## Check updatedness of tmDict when we change tm

In [9]:
print(tm[2][3]['loc'])
print(tmDict[2][3]['loc'])

0.6
0.6


In [10]:
tm.set_bound_loc((2,3), 0.6)
print(tm[2][3]['loc'])
print(tmDict[2][3]['loc'])

0.6
0.6


In [11]:
print(tmDict[2][3]['velocities'])

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]


In [12]:
tm[2][3]['velocities'] = np.ones(len(tm[2][3]['velocities'] ))

In [13]:
print(tmDict[2][3]['velocities'])

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]


# Checkpointing the dictionaries


The `cp.saveDicts` method writes any dictionaries to the `cp.savepath`

In [14]:
step = 0
time = 0.0

cp.saveDicts(step, time)